<a href="https://colab.research.google.com/github/destj-hue/edmonton-property-analysis/blob/main/YEG_Property_Assessments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show, output_notebook, save
from bokeh.models import HoverTool, ColorBar, LinearColorMapper, ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256, Category20
from bokeh.layouts import column
output_notebook()

# Load the data
df = pd.read_excel('Property_Assessment_Data.xlsx')
print(df.head())

# Display column names and types
print("\nColumn names and data types:")
print(df.dtypes)
print("\nColumn names:")
print(df.columns.tolist())

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Identify the assessed value column (check actual column name)
# Common names: 'assessed_value', 'total_assessed_value', 'Assessed Value'
value_col = None
for col in df.columns:
    if 'assess' in col.lower() and 'value' in col.lower():
        value_col = col
        print(f"\nFound assessed value column: '{value_col}'")
        break

if value_col is None:
    print("\nPlease identify the assessed value column name:")
    print(df.columns.tolist())
    value_col = 'assessed_value'

# Convert other numeric columns
if 'Latitude' in df.columns:
    df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')

if 'Longitude' in df.columns:
    df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Remove outliers (properties with assessed value = 0 or negative)
df['assessed_value'] = pd.to_numeric(df['Assessed Value'], errors='coerce')

# Remove rows with missing assessed values
df = df.dropna(subset=['assessed_value'])
df = df[df['assessed_value'] > 0]

# Residential Tax Class Filter
tax_class_col = None
for col in df.columns:
    if 'tax' in col.lower() and 'class' in col.lower():
        tax_class_col = col
        break
print(df[tax_class_col].value_counts())
df_residential = df[df[tax_class_col] == 'Residential'].copy()
df = df_residential
print(f"Residential properties: {len(df_residential)}")
print(f"Percentage of total: {len(df_residential)/len(df)*100:.1f}%")

print(f"\nCleaned data shape: {df.shape}")
print(f"Assessed value range: ${df['assessed_value'].min():,.2f} to ${df['assessed_value'].max():,.2f}")

mean_value = df['assessed_value'].mean()
median_value = df['assessed_value'].median()
std_value = df['assessed_value'].std()
min_value = df['assessed_value'].min()
max_value = df['assessed_value'].max()

print(f"\nMean Assessed Value: ${mean_value:,.2f}")
print(f"Median Assessed Value: ${median_value:,.2f}")
print(f"Standard Deviation: ${std_value:,.2f}")
print(f"Minimum Value: ${min_value:,.2f}")
print(f"Maximum Value: ${max_value:,.2f}")

print(df[df['assessed_value'] == 500])

# Quartiles and Percentiles
quartiles = df['assessed_value'].quantile([0.25, 0.5, 0.75])
percentiles = df['assessed_value'].quantile([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

print("\nQuartiles:")
print(f"Q1 (25th percentile): ${quartiles[0.25]:,.2f}")
print(f"Q2 (50th percentile/Median): ${quartiles[0.5]:,.2f}")
print(f"Q3 (75th percentile): ${quartiles[0.75]:,.2f}")

print("\nKey Percentiles:")
for pct in [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]:
    print(f"{int(pct*100)}th percentile: ${percentiles[pct]:,.2f}")

# Select only numeric columns for correlation
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Calculate correlations with assessed_value
correlations = df[numeric_cols].corr()['assessed_value'].sort_values(ascending=False)

print("\nCorrelation of all numeric factors with Assessed Value:")
print("(1.0 = perfect positive correlation, -1.0 = perfect negative correlation, 0 = no correlation)")
print("\n")
for col, corr_value in correlations.items():
    if col != 'assessed_value':  # Skip self-correlation
        print(f"{col:30s}: {corr_value:6.3f}")

# Identify strongest correlations (excluding self)
correlations_no_self = correlations.drop('assessed_value')
strongest_positive = correlations_no_self.nlargest(5)
strongest_negative = correlations_no_self.nsmallest(5)

print("TOP 5 STRONGEST POSITIVE CORRELATIONS:")
for col, corr_value in strongest_positive.items():
    print(f"{col:30s}: {corr_value:6.3f}")

print("TOP 5 STRONGEST NEGATIVE CORRELATIONS:")
for col, corr_value in strongest_negative.items():
    print(f"{col:30s}: {corr_value:6.3f}")

# Identify the property type column (may vary in dataset)
property_type_col = None
for col in df.columns:
    col_lower = col.lower()
    if 'mill' in col_lower and 'class' in col_lower:
        property_type_col = col
        break
    elif 'property' in col_lower and 'type' in col_lower:
        property_type_col = col
        break
    elif 'assessment' in col_lower and 'class' in col_lower:
        property_type_col = col
        break

if property_type_col:
    print(f"\nUsing property type column: '{property_type_col}'")

# Group by property type
    property_stats = df.groupby(property_type_col)['assessed_value'].agg([
      ('count', 'count'),
      ('mean', 'mean'),
      ('median', 'median'),
      ('std', 'std'),
      ('min', 'min'),
      ('max', 'max')
    ]).round(2)

    print(f"\nStatistics by {property_type_col}:")
    print(property_stats)

# Calculate proportion of each property type
    print("\nProportion of Each Property Type:")
    proportions = df[property_type_col].value_counts(normalize=True) * 100
    print(proportions.round(2))

# Identify property types with highest variability (coefficient of variation)
    print("\nProperty Types Ranked by Variability (Coefficient of Variation):")
    cv = (property_stats['std'] / property_stats['mean']) * 100
    cv_sorted = cv.sort_values(ascending=False)
    print(cv_sorted.round(2))
else:
    print("\nProperty type column not found. Available columns:")
    print(df.columns.tolist())

# Identify neighborhood/ward column
print(f"DataFrame shape: {df.shape}")
print(f"'Neighbourhood' in columns? {'Neighbourhood' in df.columns}")
print("\nAll columns after filtering:")
for i, col in enumerate(df.columns):
    print(f"{i}: '{col}'")
print("\n")

# Check for Neighbourhood column
if 'Neighbourhood' in df.columns:
    geo_col = 'Neighbourhood'
elif 'neighbourhood' in df.columns:
    geo_col = 'neighbourhood'
else:
    # Try to find it by searching column names
    geo_col = None
    for col in df.columns:
        if 'neighbour' in col.lower():
            geo_col = col
            break
    if geo_col is None:
        print("Warning: Neighbourhood column not found!")
        print("Available columns:", df.columns.tolist())
        geo_col = 'Neighbourhood'  # Default fallback

# Check for Ward column
if 'Ward' in df.columns:
    ward = 'Ward'
elif 'ward' in df.columns:
    ward = 'ward'
else:
    ward = None
    for col in df.columns:
        if 'ward' in col.lower():
            ward = col
            break
    if ward is None:
        print("Warning: Ward column not found!")
        ward = 'Ward'  # Default fallback

print(f"'Neighbourhood' in columns? {geo_col in df.columns}")
print(f"'Ward' in columns? {ward in df.columns}")

# Calculate average assessed values by neighborhood
if geo_col in df.columns:
    neighbourhood_stats = df.groupby(geo_col)['assessed_value'].agg([
        ('count', 'count'),
        ('mean', 'mean'),
        ('median', 'median')
    ]).round(2)

    neighbourhood_stats = neighbourhood_stats.sort_values('mean', ascending=False)

    print(f"\nTop 10 Most Expensive Neighbourhoods (by mean value):")
    print(neighbourhood_stats.head(10))

    print(f"\nTop 10 Least Expensive Neighbourhoods (by mean value):")
    print(neighbourhood_stats.tail(10))
else:
    print(f"\nSkipping neighbourhood analysis - column '{geo_col}' not found")

# Calculate average assessed values by ward
if ward in df.columns:
    ward_stats = df.groupby(ward)['assessed_value'].agg([
        ('count', 'count'),
        ('mean', 'mean'),
        ('median', 'median')
    ]).round(2)

    ward_stats = ward_stats.sort_values('mean', ascending=False)

    print(f"\nMost Expensive Ward (by mean value):")
    print(ward_stats.head(1))

    print(f"\nLeast Expensive Ward (by mean value):")
    print(ward_stats.tail(1))
else:
    print(f"\nSkipping ward analysis - column '{ward}' not found")

# Create value brackets
brackets = [0, 200000, 400000, 600000, 800000, 1000000, float('inf')]
labels = ['<$200K', '$200K-400K', '$400K-600K', '$600K-800K', '$800K-1M', '>$1M']

df['value_bracket'] = pd.cut(df['assessed_value'], bins=brackets, labels=labels)

# Count properties in each bracket
bracket_counts = df['value_bracket'].value_counts().sort_index()
print("\nProperties by Value Bracket:")
print(bracket_counts)

# Analyze characteristics by bracket
print("\nCharacteristics by Value Bracket:")
bracket_analysis = df.groupby('value_bracket').agg({
    'assessed_value': ['count', 'mean', 'median']
})
print(bracket_analysis)

# VISUALIZATION 1: Interactive Histogram - Distribution of Assessed Values
hist, edges = np.histogram(df['assessed_value'], bins=50)

p1 = figure(title='Distribution of Property Assessed Values',
           x_axis_label='Assessed Value ($)',
           y_axis_label='Frequency',
           width=800, height=400,
           x_range=(0, 2500000))

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color='navy', line_color='white', alpha=0.7)

from bokeh.models import NumeralTickFormatter
p1.xaxis.ticker = [0, 250000, 500000, 750000, 1000000, 1250000, 1500000, 1750000, 2000000, 2250000, 2500000]
p1.xaxis.formatter = NumeralTickFormatter(format="$0,0")
p1.yaxis.ticker = [0, 50000, 100000, 150000, 200000, 250000, 300000]
p1.yaxis.formatter = NumeralTickFormatter(format="0,0")

p1.add_tools(HoverTool(tooltips=[('Range', '@left{0,0} - @right{0,0}'),
                                  ('Count', '@top')]))

show(p1)

# VISUALIZATION 2: Interactive Bar Chart - Average Values by Neighbourhood
if geo_col:
    # Get top 20 neighbourhoods by mean value
    top_neighbourhoods = neighbourhood_stats.head(20).reset_index()

    source = ColumnDataSource(top_neighbourhoods)

    p2 = figure(x_range=top_neighbourhoods[geo_col].tolist(),
               title='Top 20 Neighbourhoods by Average Assessed Value',
               x_axis_label='Neighbourhood',
               y_axis_label='Average Assessed Value ($)',
               width=1000, height=500)

    p2.vbar(x=geo_col, top='mean', width=0.8, source=source,
            line_color='white', fill_color='green', alpha=0.8)

    p2.xaxis.major_label_orientation = 45

    from bokeh.models import NumeralTickFormatter
    p2.yaxis.ticker = [0, 5000000, 10000000, 15000000, 20000000]
    p2.yaxis.formatter = NumeralTickFormatter(format="$0,0")

    p2.add_tools(HoverTool(tooltips=[('Neighbourhood', f'@{geo_col}'),
                                      ('Avg Value', '@mean{$0,0}'),
                                      ('Count', '@count')]))

    show(p2)

# VISUALIZATION 3: Interactive Bar Chart - Average Values by Ward
if ward and ward in df.columns:
    # Get top 5 wards by mean value
    top_wards = ward_stats.head(5).reset_index()

    source = ColumnDataSource(top_wards)

    p3 = figure(x_range=top_wards[ward].tolist(),
               title='Top 5 Wards by Average Assessed Value',
               x_axis_label='Ward',
               y_axis_label='Average Assessed Value ($)',
               width=1000, height=500)

    p3.vbar(x=ward, top='mean', width=0.8, source=source,
            line_color='white', fill_color='green', alpha=0.8)

    p3.xaxis.major_label_orientation = 45

    from bokeh.models import NumeralTickFormatter
    p3.yaxis.ticker = [0, 100000, 200000, 300000, 400000, 500000]
    p3.yaxis.formatter = NumeralTickFormatter(format="$0,0")

    p3.add_tools(HoverTool(tooltips=[('Ward', f'@{ward}'),
                                      ('Avg Value', '@mean{$0,0}'),
                                      ('Count', '@count')]))

    show(p3)

# VISUALIZATION 4: Interactive Heatmap - Geographic Distribution
print("Has Latitude?", 'Latitude' in df.columns)
print("Has Longitude?", 'Longitude' in df.columns)
from bokeh.models import NumeralTickFormatter

if 'Latitude' in df.columns and 'Longitude' in df.columns:
    # Remove missing coordinates
    df_map = df.dropna(subset=['Latitude', 'Longitude'])

    # Create color mapper for assessed values
    mapper = linear_cmap(field_name='assessed_value',
                        palette=Viridis256,
                        low=df_map['assessed_value'].quantile(0.05),
                        high=df_map['assessed_value'].quantile(0.95))

    source = ColumnDataSource(df_map)

    p4 = figure(title='Geographic Distribution of Property Values',
               x_axis_label='Longitude',
               y_axis_label='Latitude',
               width=900, height=700,
               tools='pan,wheel_zoom,box_zoom,reset')

    p4.circle('Longitude', 'Latitude', source=source,
             size=3, alpha=0.6, color=mapper)

    color_bar = ColorBar(color_mapper=mapper['transform'],
                        width=8, location=(0,0),
                        title="Assessed Value ($)",
                        formatter=NumeralTickFormatter(format="$0,0"))
    p4.add_layout(color_bar, 'right')

    p4.add_tools(HoverTool(tooltips=[('Location', '@latitude{0.0000}, @longitude{0.0000}'),
                                      ('Value', '@assessed_value{$0,0}')]))

    show(p4)

#VISUALIZATION 5: Correlation
# Get top 10 correlations (positive and negative)
top_10_corr = pd.concat([
    strongest_positive.head(5),
    strongest_negative.head(5)
]).sort_values().drop_duplicates()

# Create bar chart
from bokeh.models import Title

# Color bars based on positive/negative
colors = ['red' if x < 0 else 'green' for x in top_10_corr.values]

source_corr = ColumnDataSource(data={
    'factors': top_10_corr.index.tolist(),
    'correlation': top_10_corr.values.tolist(),
    'colors': colors
})

p_corr = figure(
    y_range=top_10_corr.index.tolist(),
    title='Top 10 Factors Correlated with Assessed Value',
    x_axis_label='Correlation Coefficient',
    y_axis_label='Factor',
    width=900,
    height=500
)

p_corr.hbar(
    y='factors',
    right='correlation',
    height=0.7,
    source=source_corr,
    color='colors',
    alpha=0.8
)

# Add a vertical line at x=0
from bokeh.models import Span
vline = Span(location=0, dimension='height', line_color='black', line_width=2)
p_corr.add_layout(vline)

p_corr.add_tools(HoverTool(tooltips=[
    ('Factor', '@factors'),
    ('Correlation', '@correlation{0.000}')
]))

subtitle = Title(text="Green = positive correlation, Red = negative correlation",
                text_font_size="10pt", text_font_style="italic")
p_corr.add_layout(subtitle, 'above')

show(p_corr)

# VISUALIZATION 6: Interactive Map - Property Values
import json

# Prepare data for map (sample for performance if dataset is huge)
if 'Latitude' in df.columns and 'Longitude' in df.columns:
    df_map = df.dropna(subset=['Latitude', 'Longitude']).copy()

    # If dataset is very large, sample it for better map performance
    if len(df_map) > 10000:
        print(f"Dataset has {len(df_map)} properties. Sampling 10,000 for map performance...")
        df_map = df_map.sample(n=10000, random_state=42)

    # Create value brackets for color coding
    df_map['value_category'] = pd.cut(
        df_map['assessed_value'],
        bins=[0, 200000, 400000, 600000, 800000, float('inf')],
        labels=['<$200K', '$200K-400K', '$400K-600K', '$600K-800K', '>$800K']
    )

    # Map categories to numeric values for color mapping
    category_to_num = {
        '<$200K': 1,
        '$200K-400K': 2,
        '$400K-600K': 3,
        '$600K-800K': 4,
        '>$800K': 5
    }
    df_map['color_value'] = df_map['value_category'].map(category_to_num)

    # Create color mapper
    from bokeh.models import CategoricalColorMapper
    color_mapper = CategoricalColorMapper(
        factors=['<$200K', '$200K-400K', '$400K-600K', '$600K-800K', '>$800K'],
        palette=['blue', 'green', 'orange', 'red', 'darkred']
    )

    # Prepare tooltip data
    df_map['value_formatted'] = df_map['assessed_value'].apply(lambda x: f"${x:,.0f}")

    # Create data source
    source = ColumnDataSource(df_map)

    # Create the map figure
    p_map = figure(
        title='Interactive Map: Edmonton Property Values',
        x_axis_label='Longitude',
        y_axis_label='Latitude',
        width=1000,
        height=700,
        tools='pan,wheel_zoom,box_zoom,reset,save'
    )

    # Add property points
    p_map.scatter(
        'Longitude',
        'Latitude',
        source=source,
        size=6,
        alpha=0.6,
        color={'field': 'value_category', 'transform': color_mapper},
        marker='circle',
        line_color='white',
        line_width=0.5
    )

    # Create hover tooltips
    tooltips = [
        ('Value', '@value_formatted'),
        ('Category', '@value_category'),
        ('Location', '(@latitude{0.0000}, @longitude{0.0000})')
    ]

    # Add neighborhood and property type to tooltips if available
    if geo_col and geo_col in df_map.columns:
        tooltips.append(('Neighborhood', f'@{geo_col}'))

    if property_type_col and property_type_col in df_map.columns:
        tooltips.append(('Property Type', f'@{property_type_col}'))

    hover = HoverTool(tooltips=tooltips)
    p_map.add_tools(hover)

    # Add title with instructions
    from bokeh.models import Title
    subtitle = Title(text="Use mouse to pan and scroll to zoom. Hover over points for details.",
                    text_font_size="10pt", text_font_style="italic")
    p_map.add_layout(subtitle, 'above')

    # Save map to HTML file
    from bokeh.plotting import output_file, save
    output_file('edmonton_map.html')
    save(p_map)
    print("✓ Map saved to edmonton_map.html")

    # Prepare data for JSON export (sample for reasonable file size)
    df_json = df_map.copy()

    # Select relevant columns for JSON
    json_columns = ['assessed_value', 'latitude', 'longitude']
    if geo_col and geo_col in df_map.columns:
        json_columns.append(geo_col)
    if property_type_col and property_type_col in df_map.columns:
        json_columns.append(property_type_col)

    # Create JSON-friendly data structure
    properties_data = []
    for idx, row in df_json.iterrows():
        property_dict = {
            'assessed_value': float(row['assessed_value']),
            'latitude': float(row['Latitude']),
            'longitude': float(row['Longitude']),
            'value_category': str(row['value_category'])
        }

        if geo_col and geo_col in row:
            property_dict['neighbourhood'] = str(row[geo_col])

        if property_type_col and property_type_col in row:
            property_dict['property_type'] = str(row[property_type_col])

        properties_data.append(property_dict)

    # Create complete JSON structure with metadata
    json_output = {
        'metadata': {
            'total_properties': len(df),
            'properties_in_export': len(properties_data),
            'mean_value': float(mean_value),
            'median_value': float(median_value),
            'min_value': float(min_value),
            'max_value': float(max_value),
            'city': 'Edmonton',
            'data_year': 2024
        },
        'properties': properties_data
    }

    # Save to JSON file
    json_filename = 'edmonton_properties.json'
    with open(json_filename, 'w') as f:
        json.dump(json_output, f, indent=2)

    # Display sample of JSON structure
    print("\nSample JSON structure:")
    print(json.dumps(json_output['properties'][0], indent=2))

# Create summary dictionary for easy reference
summary = {
    'Total Properties': len(df),
    'Mean Value': f"${mean_value:,.2f}",
    'Median Value': f"${median_value:,.2f}",
    'Std Deviation': f"${std_value:,.2f}",
    'Min Value': f"${min_value:,.2f}",
    'Max Value': f"${max_value:,.2f}"
}

print("\nSummary Statistics:")
for key, value in summary.items():
    print(f"{key}: {value}")